In [1]:
import sys
import cv2
import numpy as np
import face_recognition
import os
import csv
from datetime import datetime
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget, QPushButton, QLineEdit,
    QHBoxLayout, QStatusBar, QMenuBar, QAction, QTableWidget, QTableWidgetItem, QMessageBox
)
from PyQt5.QtGui import QImage, QPixmap, QIcon
from PyQt5.QtCore import QTimer, Qt
from stylesheet import get_stylesheet  # Import the stylesheet function

class FaceRecognitionApp(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Face Recognition Attendance System")
        self.setGeometry(100, 100, 1000, 800)  # Increased initial size for better layout
        self.setWindowIcon(QIcon("icons/gojo satoru.png"))  # Set your application icon here

        # Menu Bar
        self.menu_bar = QMenuBar(self)
        self.setMenuBar(self.menu_bar)
        
        file_menu = self.menu_bar.addMenu("File")
        help_menu = self.menu_bar.addMenu("Help")
        
        exit_action = QAction("Exit", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)
        
        about_action = QAction("About", self)
        about_action.triggered.connect(self.show_about_dialog)
        help_menu.addAction(about_action)

        # Status Bar
        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

        self.image_label = QLabel(self)
        self.image_label.setAlignment(Qt.AlignCenter)

        self.name_input = QLineEdit(self)
        self.name_input.setPlaceholderText("Enter your name")

        self.capture_button = QPushButton("Capture Photo", self)
        self.capture_button.setIcon(QIcon("icons/camera.png"))  # Set your button icon here
        self.capture_button.clicked.connect(self.capture_photo)

        self.view_logs_button = QPushButton("View Attendance Logs", self)
        self.view_logs_button.setIcon(QIcon("icons/file manager.png"))  # Set your button icon here
        self.view_logs_button.clicked.connect(self.view_logs)

        self.date_time_label = QLabel(self)
        self.date_time_label.setAlignment(Qt.AlignCenter)
        self.date_time_label.setStyleSheet("font-size: 14px; color: white; padding: 5px;")  # Customize as needed
        self.date_time_label.setFixedHeight(30)  # Set a fixed height for the date time label

        self.layout = QVBoxLayout()
        self.layout.addWidget(self.date_time_label)  # Add the date time label at the top
        self.layout.addWidget(self.image_label)

        input_layout = QHBoxLayout()
        input_layout.addWidget(self.name_input)
        input_layout.addWidget(self.capture_button)
        input_layout.addWidget(self.view_logs_button)
        self.layout.addLayout(input_layout)

        self.container = QWidget()
        self.container.setLayout(self.layout)
        self.setCentralWidget(self.container)

        self.cap = cv2.VideoCapture(0)
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(20)

        self.path = 'test_images'
        self.images = []
        self.classNames = []
        self.load_images()
        self.encodeListKnown = self.find_encodings(self.images)
        print('Encoding Complete')

        # Timer to update date and time label
        self.date_timer = QTimer()
        self.date_timer.timeout.connect(self.update_date_time)
        self.date_timer.start(1000)  # Update every second

        # Apply style sheet
        self.apply_stylesheet()

    def apply_stylesheet(self):
        self.setStyleSheet(get_stylesheet())  # Apply the stylesheet from stylesheet.py

    def load_images(self):
        myList = os.listdir(self.path)
        for cl in myList:
            curImg = cv2.imread(f'{self.path}/{cl}')
            self.images.append(curImg)
            self.classNames.append(os.path.splitext(cl)[0])

    def find_encodings(self, images):
        encodeList = []
        for img in images:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            encode = face_recognition.face_encodings(img)
            if encode:
                encodeList.append(encode[0])
        return encodeList

    def mark_attendance(self, name):
        current_date = datetime.now().strftime('%Y-%m-%d')
        data_exists = False
        
        # Read existing data
        try:
            with open('data.csv', 'r') as f:
                reader = csv.reader(f)
                rows = list(reader)
        except FileNotFoundError:
            rows = []
        
        # Check if there's an entry for the current date for the given name
        for row in rows:
            if len(row) >= 2 and row[0] == name and row[1].startswith(current_date):
                data_exists = True
                break

        # If no entry exists for the current date, add it
        if not data_exists:
            now = datetime.now()
            dtString = now.strftime('%Y-%m-%d %H:%M:%S')
            with open('data.csv', 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([name, dtString])
            self.status_bar.showMessage(f"Attendance marked for {name} at {dtString}", 5000)  # Show message for 5 seconds

    def capture_photo(self):
        success, img = self.cap.read()
        if success:
            name = self.name_input.text().strip()
            if name:
                img_path = os.path.join(self.path, f"{name}.jpg")
                cv2.imwrite(img_path, img)
                self.images.append(img)
                self.classNames.append(name)
                self.encodeListKnown = self.find_encodings(self.images)
                self.status_bar.showMessage(f"Photo captured and saved as {img_path}", 5000)  # Show message for 5 seconds
            else:
                self.status_bar.showMessage("Name cannot be empty", 5000)  # Show message for 5 seconds

    def update_frame(self):
        success, img = self.cap.read()
        if success:
            imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
            imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

            facesCurFrame = face_recognition.face_locations(imgS)
            encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

            face_recognized = False

            for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
                matches = face_recognition.compare_faces(self.encodeListKnown, encodeFace, tolerance=0.5)  # Decreased tolerance for stricter matching
                faceDis = face_recognition.face_distance(self.encodeListKnown, encodeFace)
                matchIndex = np.argmin(faceDis)

                if matches[matchIndex]:
                    face_recognized = True
                    name = self.classNames[matchIndex].upper()
                    y1, x2, y2, x1 = faceLoc
                    y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                    cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                    self.mark_attendance(name)

            if not face_recognized:
                cv2.putText(img, "Face not recognized", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                
            self.display_image(img)

    def display_image(self, img):
        qformat = QImage.Format_RGB888
        img = QImage(img, img.shape[1], img.shape[0], img.strides[0], qformat)
        img = img.rgbSwapped()
        self.image_label.setPixmap(QPixmap.fromImage(img))
        self.image_label.setScaledContents(True)

    def update_date_time(self):
        now = datetime.now()
        dtString = now.strftime('%Y-%m-%d %H:%M:%S')
        self.date_time_label.setText(dtString)

    def view_logs(self):
        self.logs_window = AttendanceLogsWindow()
        self.logs_window.show()

    def show_about_dialog(self):
        about_message = "Face Recognition Attendance System\nVersion 1.0\nDeveloped by NitinRawat"
        QMessageBox.about(self, "About", about_message)


class AttendanceLogsWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Attendance Logs")
        self.setGeometry(200, 200, 400, 300)
        self.setWindowIcon(QIcon("icons/admin.png"))  # Set your window icon here

        self.layout = QVBoxLayout()
        self.setLayout(self.layout)

        self.table_widget = QTableWidget()
        self.layout.addWidget(self.table_widget)

        self.load_data()

    def load_data(self):
        try:
            with open('data.csv', 'r') as f:
                reader = csv.reader(f)
                rows = list(reader)
                self.table_widget.setRowCount(len(rows))
                self.table_widget.setColumnCount(2)
                self.table_widget.setHorizontalHeaderLabels(["Name", "Date and Time"])

                for row_index, row_data in enumerate(rows):
                    for col_index, data in enumerate(row_data):
                        self.table_widget.setItem(row_index, col_index, QTableWidgetItem(data))
        except FileNotFoundError:
            self.table_widget.setRowCount(0)
            self.table_widget.setColumnCount(2)
            self.table_widget.setHorizontalHeaderLabels(["Name", "Date and Time"])


if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = FaceRecognitionApp()
    main_win.show()
    sys.exit(app.exec_())


Encoding Complete


SystemExit: 0

c:\Users\soura\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
